In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '60_memory_and_registry'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Lab 60 · Memory and Registry (Dry-Run)

The memory and registry modules depend on external tooling such as
Volatility and RegRipper. This lab focuses on dry-run guards and deterministic
metadata so analysts can rehearse workflows even when those tools are missing.


## Tool Guards
Before running any heavy analysis we confirm whether the expected binaries
are present. Volatility is required for memory parsing, while RegRipper is an
optional enhancer for Windows registry hives.


In [ ]:
from forensic.tools import volatility as volatility_tools

VOL_OK = volatility_tools.available()
VOL_VERSION = volatility_tools.version() if VOL_OK else None
REG_OK = shell_available('rip.pl')

TOOLS_STATUS = {
    'volatility_available': VOL_OK,
    'volatility_version': VOL_VERSION,
    'volatility_requirements': volatility_tools.requirements(),
    'regripper_available': REG_OK,
    'regripper_hint': 'Install RegRipper (rip.pl) for extended hive parsing' if not REG_OK else 'RegRipper detected',
}
json_dump_sorted(TOOLS_STATUS, LAB_ROOT / 'tool_status.json')
TOOLS_STATUS


### Guard Interpretation
If Volatility or RegRipper are missing we surface human-friendly guidance
instead of failing. The wrappers remain callable in dry-run mode so pipelines
can be rehearsed without privileged binaries.


In [ ]:
from forensic.modules.analysis.memory import MemoryAnalysisModule
from forensic.modules.analysis.registry import RegistryAnalysisModule

CASE_DIR = LAB_ROOT / 'case_memory_registry'
CASE_DIR.mkdir(parents=True, exist_ok=True)

memory_module = MemoryAnalysisModule(case_dir=CASE_DIR, config={})
registry_module = RegistryAnalysisModule(case_dir=CASE_DIR, config={})

module_versions = {
    'memory_tool_versions': memory_module.tool_versions(),
    'registry_tool_versions': registry_module.tool_versions(),
}
json_dump_sorted(module_versions, LAB_ROOT / 'module_versions.json')
module_versions


## Parameter Validation (Dry-Run)
Even without the external tools we can validate parameters. Passing a missing
dump or hive path results in a deterministic guard instead of a traceback.


In [ ]:
invalid_dump = CASE_DIR / 'missing.mem'
invalid_hive = CASE_DIR / 'registry' / 'sam'

memory_valid = memory_module.validate_params({'dump': str(invalid_dump)})
registry_valid = registry_module.validate_params({'hives': [str(invalid_hive)]})

validation_report = {
    'memory_validation': memory_valid,
    'registry_validation': registry_valid,
    'memory_hint': 'Provide an existing memory image' if not memory_valid else 'OK',
    'registry_hint': 'Supply at least one hive file (SAM, SYSTEM, SOFTWARE, ...)' if not registry_valid else 'OK',
}
json_dump_sorted(validation_report, LAB_ROOT / 'validation_report.json')
validation_report


## Synthetic Memory Metadata
A deterministic JSON payload mirrors the summaries Volatility would produce.
It references process, network, and heuristic findings so that downstream
notebooks can exercise parsing logic without a real dump.


In [ ]:
MEMORY_TS = _ts()
MEMORY_DIR = LAB_ROOT / 'memory'
MEMORY_DIR.mkdir(parents=True, exist_ok=True)

memory_snapshot = {
    'module': 'memory_analysis',
    'generated_at': MEMORY_TS,
    'source_image': 'synthetic-workstation.mem',
    'volatility_available': TOOLS_STATUS['volatility_available'],
    'processes': [
        {'pid': 4, 'name': 'System', 'ppid': 0, 'command': 'System', 'integrity': 'system'},
        {'pid': 1337, 'name': 'beacon.exe', 'ppid': 432, 'command': 'C\\Tools\\beacon.exe', 'integrity': 'medium'},
    ],
    'network': [
        {'pid': 1337, 'laddr': '10.0.0.5:49832', 'raddr': '198.51.100.23:443', 'state': 'ESTABLISHED'},
        {'pid': 556, 'laddr': '10.0.0.5:22', 'raddr': '203.0.113.42:55210', 'state': 'CLOSE_WAIT'},
    ],
    'malware_indicators': [
        {'plugin': 'malfind', 'pid': 1337, 'score': 0.92, 'notes': 'Injected PE section with RWX permissions'},
    ],
}

memory_json_path = json_dump_sorted(memory_snapshot, MEMORY_DIR / f'{MEMORY_TS}_memory_snapshot.json')
preview(memory_json_path)


## Synthetic Registry Metadata
We capture a minimal registry state describing autoruns and network settings.
When RegRipper is unavailable the payload records that fact for analysts.


In [ ]:
REGISTRY_TS = _ts()
REGISTRY_DIR = LAB_ROOT / 'registry'
REGISTRY_DIR.mkdir(parents=True, exist_ok=True)

registry_snapshot = {
    'module': 'registry_analysis',
    'generated_at': REGISTRY_TS,
    'hives': {
        'SAM': {'path': 'synthetic/SAM', 'found': False},
        'SYSTEM': {'path': 'synthetic/SYSTEM', 'found': True, 'last_boot': '2024-03-01T11:58:00Z'},
        'SOFTWARE': {
            'path': 'synthetic/SOFTWARE',
            'run_keys': [
                {'name': 'PersistenceAgent', 'value': 'C:/ProgramData/pa.exe', 'source': 'HKLM\\Software\\Microsoft\\Windows\\CurrentVersion\\Run'},
            ],
        },
    },
    'regripper_available': TOOLS_STATUS['regripper_available'],
    'guidance': TOOLS_STATUS['regripper_hint'],
}

registry_json_path = json_dump_sorted(registry_snapshot, REGISTRY_DIR / f'{REGISTRY_TS}_registry_snapshot.json')
preview(registry_json_path)


## Module Pathing Demonstration
When the real modules execute they populate `analysis/memory_analysis/` and
`analysis/registry_analysis/`. We mimic this layout so follow-up notebooks can
consume the synthetic artefacts.


In [ ]:
MEMORY_OUTPUT = CASE_DIR / 'analysis' / memory_module.name
REGISTRY_OUTPUT = CASE_DIR / 'analysis' / registry_module.name
MEMORY_OUTPUT.mkdir(parents=True, exist_ok=True)
REGISTRY_OUTPUT.mkdir(parents=True, exist_ok=True)

memory_copy = MEMORY_OUTPUT / memory_json_path.name
registry_copy = REGISTRY_OUTPUT / registry_json_path.name

memory_copy.write_text((LAB_ROOT / 'memory' / memory_json_path.name).read_text(encoding='utf-8'), encoding='utf-8')
registry_copy.write_text((LAB_ROOT / 'registry' / registry_json_path.name).read_text(encoding='utf-8'), encoding='utf-8')

path_report = {
    'memory_output_dir': str(MEMORY_OUTPUT),
    'registry_output_dir': str(REGISTRY_OUTPUT),
    'memory_artifact': str(memory_copy),
    'registry_artifact': str(registry_copy),
}
json_dump_sorted(path_report, LAB_ROOT / 'path_report.json')
path_report


### Checkpoint
Synthetic artefacts are now in place even if external tooling is missing.


In [ ]:
expected = [
    LAB_ROOT / 'tool_status.json',
    memory_json_path,
    registry_json_path,
    memory_copy,
    registry_copy,
]
for item in expected:
    assert Path(item).exists(), f'Missing artefact: {item}'

GUARD_STATUS = {
    'tools_status': str(LAB_ROOT / 'tool_status.json'),
    'memory_snapshot': str(memory_json_path),
    'registry_snapshot': str(registry_json_path),
}
json_dump_sorted(GUARD_STATUS, LAB_ROOT / 'guard_status.json')
GUARD_STATUS
